In [1]:
# add autoreload
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import transformers
import torch

import os
import sys

from setfit import SetFitModel
from setfit import SetFitTrainer
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score

from transformers import TrainingArguments, Trainer
from transformers import pipeline
from transformers import DataCollatorWithPadding
from transformers import EvalPrediction

from torchinfo import summary
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import evaluate

from sklearn.preprocessing import LabelEncoder
import gc

In [2]:
plt.style.use('ggplot')
def plot_history(history, val=0):
    acc = history.history['accuracy']
    if val == 1:
        val_acc = history.history['val_accuracy'] # we can add a validation set in our fit function with nn
    loss = history.history['loss']
    if val == 1:
        val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training accuracy')
    if val == 1:
        plt.plot(x, val_acc, 'r', label='Validation accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.title('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    if val == 1:
        plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.title('Loss')
    plt.legend()

In [3]:
def compute_metrics_binomial(logits_and_labels, averaging='macro'):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average = averaging)
  prec = precision_score(labels, predictions, average = averaging)
  rec = recall_score(labels, predictions, average = averaging)
  return {
          'accuracy': acc, 
          'f1_score': f1,
          'precision': prec,
          'recal': rec
          }

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    
    
    f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    f1_weighted = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
    prec_macro = precision_score(y_true=y_true, y_pred=y_pred, average='macro')
    prec_weighted = precision_score(y_true=y_true, y_pred=y_pred, average='weighted')
    recall_macro = recall_score(y_true=y_true, y_pred=y_pred, average='macro')
    recall_weighted = recall_score(y_true=y_true, y_pred=y_pred, average='weighted')    
    
    try:
        roc_auc_weighted = roc_auc_score(y_true, probs, average = 'weighted')
        roc_auc_macro = roc_auc_score(y_true, probs, average = 'macro')
    except:
        roc_auc_weighted = None
        roc_auc_macro = None
    
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1_macro': f1_macro,
               'f1_weighted': f1_weighted,
               'prec_macro': prec_macro,
               'prec_weighted': prec_weighted,
               'recall_macro': recall_macro,
               'recall_weighted': recall_weighted,
               'roc_auc_macro': roc_auc_macro,
               'roc_auc_weighted': roc_auc_weighted,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result
     


In [4]:
def one_hot_encode(labels, num_classes):
    # labels should be a list or a 1D tensor
    one_hot = torch.zeros((len(labels), num_classes))
    rows = torch.arange(len(labels))
    one_hot[rows, labels] = 1
    return one_hot

# Load documents

In [5]:
tokenizer = AutoTokenizer.from_pretrained("CLTL/MedRoBERTa.nl")

In [6]:
os.chdir('T://lab_research/RES-Folder-UPOD/Echo_label/E_ResearchData/2_ResearchData')

In [7]:
labeled_documents = pd.read_json("./echo_doc_labels/merged_labels.jsonl", lines=True)
target_df = pd.DataFrame.from_records(labeled_documents['labels'])

In [8]:
labeled_documents.head()

,text,labels,_input_hash
0,Matige cLVH (toegenomen tov echo 2006). Goede ...,"{'lv_sys_func': 'Normal', 'lv_dil': 'No_label'...",868867415
1,Goede LV functie. LA matig gedilateerd. Normal...,"{'lv_sys_func': 'Normal', 'lv_dil': 'No_label'...",-1892706589
2,Aneurysmatisch interatriaal septum. Overig nor...,"{'lv_sys_func': 'Normal', 'lv_dil': 'Normal', ...",1169657656
3,Alleen apicaal venster. Normale dimensies. Goe...,"{'lv_sys_func': 'Normal', 'lv_dil': 'Normal', ...",2132812484
4,"Irr. ritme! (zeer freq VES) Matige syst. LVF, ...","{'lv_sys_func': 'Moderate', 'lv_dil': 'No_labe...",-2123317975


In [9]:
# Expand with label columns
Target_maps = {
    Class: {Label:i for i,Label in enumerate(target_df[Class].unique())}
        for Class in target_df.columns
}

In [10]:
# Load the train/test hashes
test_hashes = pd.read_csv('./test_echoid.csv', sep=',')
train_hashes = pd.read_csv('./train_echoid.csv', sep=',')
print(train_hashes.columns)

Index(['ECHO_StudyID', 'ECHO_StudyID.1', 'input_hash', 'task_hash'], dtype='object')


In [11]:
# We now make DataSets (a special HuggingFace structure)
# assuming cross-validation

Class = 'pe' 

DF = labeled_documents[['text', '_input_hash']].join(target_df[Class])
DF.columns = ['sentence', '_input_hash', 'labels']

label2id = Target_maps[Class]
id2label = {v:k for k,v in label2id.items()}
num_labels = len(label2id)

DF['labels'] = DF['labels'].map(label2id)

# TODO: make proper
DFtrain = DF.loc[DF._input_hash.isin(train_hashes.input_hash), ['sentence', 'labels']]
DFtest = DF.loc[DF._input_hash.isin(test_hashes.input_hash), ['sentence', 'labels']]

TrainSet = Dataset.from_pandas(DFtrain)
TestSet = Dataset.from_pandas(DFtest)

HF_DataSet = DatasetDict(
    {'train' : TrainSet,
     'test': TestSet,
    }
)

Tokenized_DataSet = HF_DataSet.map(lambda batch: tokenizer(batch, truncation=True, 
                                                                  padding=True, 
                                                                  max_length=256),
                                  input_columns='sentence',
                                  batched=True,
                                  remove_columns=['sentence'])

Tokenized_DataSet = (Tokenized_DataSet
                      #.map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
                      .map(lambda x: {"labels": one_hot_encode(x["labels"],
                                                               num_classes=num_labels)}, 
                           batched=True, remove_columns=['labels']))                      
                      #.rename_column("float_labels", "labels"))

Tokenized_DataSet.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/4031 [00:00<?, ? examples/s]

Map:   0%|          | 0/991 [00:00<?, ? examples/s]

Map:   0%|          | 0/4031 [00:00<?, ? examples/s]

Map:   0%|          | 0/991 [00:00<?, ? examples/s]

In [12]:
# https://colab.research.google.com/drive/1aue7x525rKy6yYLqqt-5Ll96qjQvpqS7#scrollTo=1eVCRpcLUW-y
# https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
medroberta_clf = AutoModelForSequenceClassification.from_pretrained("CLTL/MedRoBERTa.nl", 
                                                num_labels=len(id2label.keys()),
                                                problem_type='multi_label_classification',
                                                id2label=id2label,
                                                label2id=label2id)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at CLTL/MedRoBERTa.nl and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from torchinfo import summary
summary(medroberta_clf)

Layer (type:depth-idx)                                       Param #
RobertaForSequenceClassification                             --
├─RobertaModel: 1-1                                          --
│    └─RobertaEmbeddings: 2-1                                --
│    │    └─Embedding: 3-1                                   39,936,000
│    │    └─Embedding: 3-2                                   394,752
│    │    └─Embedding: 3-3                                   768
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─RobertaEncoder: 2-2                                   --
│    │    └─ModuleList: 3-6                                  85,054,464
├─RobertaClassificationHead: 1-2                             --
│    └─Linear: 2-3                                           590,592
│    └─Dropout: 2-4                                          --
│    └─Linear: 2-5                                           4,614
To

## MedRoBERTa.nl -- Training setup

In [14]:
train_dir = "T://laupodteam/AIOS/Bram/data/tmp"
metric_name = 'f1_macro'
training_args = TrainingArguments(output_dir=train_dir,
                                  evaluation_strategy='epoch',
                                  save_strategy='epoch',
                                  num_train_epochs=20,
                                  learning_rate=5e-5,
                                  per_device_train_batch_size=16,
                                  weight_decay=0.01,
                                  per_device_eval_batch_size=10,
                                  load_best_model_at_end=True, 
                                  metric_for_best_model=metric_name)

In [15]:
trainer = Trainer(medroberta_clf,
                  training_args,
                  train_dataset = Tokenized_DataSet["train"],
                  eval_dataset = Tokenized_DataSet["test"],
                  tokenizer=tokenizer,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics)

c:\Users\bes3\VIRTUALENVS\nlp\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## MedRoBERTa.nl -- run Training

In [16]:
trainer.train()

  0%|          | 0/5040 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\bes3\VIRTUALENVS\nlp\lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\bes3\VIRTUALENVS\nlp\lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\bes3\VIRTUALENVS\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bes3\VIRTUALENVS\nlp\l

{'eval_loss': 0.04280763491988182, 'eval_f1_macro': 0.3093906786605623, 'eval_f1_weighted': 0.9472625862991825, 'eval_prec_macro': 0.29618286502873176, 'eval_prec_weighted': 0.9368485467894367, 'eval_recall_macro': 0.32543167583653415, 'eval_recall_weighted': 0.9596367305751766, 'eval_roc_auc_macro': None, 'eval_roc_auc_weighted': None, 'eval_accuracy': 0.9596367305751766, 'eval_runtime': 10.4619, 'eval_samples_per_second': 94.724, 'eval_steps_per_second': 9.558, 'epoch': 1.0}
{'loss': 0.0767, 'grad_norm': 2.1688852310180664, 'learning_rate': 2.7023809523809527e-05, 'epoch': 1.98}


  0%|          | 0/100 [00:00<?, ?it/s]

...

{'eval_loss': 0.033551961183547974, 'eval_f1_macro': 0.5491683097182932, 'eval_f1_weighted': 0.9747816363164237, 'eval_prec_macro': 0.575424836874628, 'eval_prec_weighted': 0.9775821070806437, 'eval_recall_macro': 0.5386569233735226, 'eval_recall_weighted': 0.9727547931382442, 'eval_roc_auc_macro': None, 'eval_roc_auc_weighted': None, 'eval_accuracy': 0.9727547931382442, 'eval_runtime': 11.3892, 'eval_samples_per_second': 87.012, 'eval_steps_per_second': 8.78, 'epoch': 19.0}
{'loss': 0.0007, 'grad_norm': 0.013616524636745453, 'learning_rate': 2.380952380952381e-07, 'epoch': 19.84}


  0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\bes3\VIRTUALENVS\nlp\lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\bes3\VIRTUALENVS\nlp\lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\bes3\VIRTUALENVS\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bes3\VIRTUALENVS\nlp\l

{'eval_loss': 0.03348866105079651, 'eval_f1_macro': 0.5491683097182932, 'eval_f1_weighted': 0.9747816363164237, 'eval_prec_macro': 0.575424836874628, 'eval_prec_weighted': 0.9775821070806437, 'eval_recall_macro': 0.5386569233735226, 'eval_recall_weighted': 0.9727547931382442, 'eval_roc_auc_macro': None, 'eval_roc_auc_weighted': None, 'eval_accuracy': 0.9727547931382442, 'eval_runtime': 11.3738, 'eval_samples_per_second': 87.13, 'eval_steps_per_second': 8.792, 'epoch': 20.0}
{'train_runtime': 3531.4234, 'train_samples_per_second': 22.829, 'train_steps_per_second': 1.427, 'train_loss': 0.011931164913056862, 'epoch': 20.0}


TrainOutput(global_step=5040, training_loss=0.011931164913056862, metrics={'train_runtime': 3531.4234, 'train_samples_per_second': 22.829, 'train_steps_per_second': 1.427, 'train_loss': 0.011931164913056862, 'epoch': 20.0})

In [17]:
trainer.eval_dataset = Tokenized_DataSet["test"]
trainer.evaluate()

  0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\bes3\VIRTUALENVS\nlp\lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\bes3\VIRTUALENVS\nlp\lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
c:\Users\bes3\VIRTUALENVS\nlp\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bes3\VIRTUALENVS\nlp\l

{'eval_loss': 0.026169225573539734,
 'eval_f1_macro': 0.6481689732945465,
 'eval_f1_weighted': 0.9789981725743834,
 'eval_prec_macro': 0.6863427372866052,
 'eval_prec_weighted': 0.981149426389365,
 'eval_recall_macro': 0.6294046259633305,
 'eval_recall_weighted': 0.9778002018163471,
 'eval_roc_auc_macro': None,
 'eval_roc_auc_weighted': None,
 'eval_accuracy': 0.9767911200807265,
 'eval_runtime': 10.9859,
 'eval_samples_per_second': 90.206,
 'eval_steps_per_second': 9.103,
 'epoch': 20.0}

In [18]:
#medroberta_clf_pipe = pipeline('text-classification', model=medroberta_clf, tokenizer=tokenizer)